In [ ]:
from awrams.calibration.support import *
from awrams.utils.nodegraph.nodes import callable_to_funcspec
from os.path import join
from awrams.utils import gis
from awrams.utils import datetools as dt
from awrams.utils import extents
from awrams.utils.io import data_mapping as dm
from awrams.utils import config_manager
import os
from awrams.utils.nodegraph import nodes

%matplotlib inline

In [ ]:
from awrams.calibration.support import CalibrationResults

In [ ]:
# To use alternative profiles, call the set_active_system_profile method
# Subsequent config_manager actions will now refer to this profile

config_manager.set_active_system_profile('raijin')

In [ ]:
# Paths etc now refer to the remote host.  Model profiles will use this information automatically to build their
# configuration (input_map etc)

config_manager.get_system_profile().get_settings().DATA_PATHS

In [ ]:
# A typical workflow would involve getting any _local_ information first using the default system profile,
# then switching the remote profile to configuration your jobs

# Clear the system profile back to default (local) and get the local paths
config_manager.set_active_system_profile()
local_settings = config_manager.get_system_profile().get_settings()

# Set the active profile again
config_manager.set_active_system_profile('raijin')
remote_settings = config_manager.get_system_profile().get_settings()

In [ ]:
# Select some catchment extents for use in this calibration run
# Note that we are loading these locally, so we use local paths

cal_extents = join(local_settings.DATA_PATHS.SHAPEFILES,'calibration_extents_5k.nc')
nces = gis.ExtentStoreNC(cal_extents,'r')

extent_map = dict([(e,nces[e]) for e in nces.available[0:6]])

sum([v.cell_count for k,v in extent_map.items()])

In [ ]:
# Because we using the training dataset, we'll limit these to the available data

run_period = dt.dates('2009-2011')
eval_period = dt.dates('2009-2011')

from awrams.calibration.optimizers import sce

# Set up our optimizer.  These are typical values for a full calibration run, so it might take some time to run!
# For testing and prototyping, you will definitely want to reduce these

evolver_spec = EvolverSpec(sce.CCEvolver,evolver_run_args=dict(n_offspring=1,n_evolutions=5,elitism=1.0))
optimizer_spec = OptimizerSpec(sce.ShuffledOptimizer,evolver_spec=evolver_spec,n_complexes=14,max_nsni=4000,min_complexes=1,max_eval=2000,init_method='lhs') #n_complex 14

# Note that our observations are being accessed on the remote system - make sure you use the right paths here
observations=dict(qtot=join(remote_settings.DATA_PATHS.BASE_DATA,'observations/runoff/awrams_v5_cal_qobs.csv'))

# For a typical workflow you will probably be editing and prototyping your objective functions on a local machine
# first.  Here we import these local functions directly, even though they will be running on the remote machine.
#
# The remote system will perform the same import logic - as long as awrams_user.examples.objectives exists in both
# places this will work correctly.  However, remember that this is not copying the _data_ in this file, merely
# informing the system of what to import
#
# We will discuss how to synchronise your user code later in the notebook

from awrams_user.objectives import example
local_objfspec = ObjectiveFunctionSpec(example.TestLocalSingle)
global_objfspec = example.TestGlobalSingle

# Collect these into the ObjectiveSpec

objective_spec = ObjectiveSpec(global_objfspec,local_objfspec,observations,eval_period)

In [ ]:
# Because we have used set_activate_system_profile, the model configuration will automatically use the remote paths
# This is the recommended method, but it is possible to set these up manually if more advanced changes are required

model_profile = config_manager.get_model_profile('awral','v6_default')
model_settings = model_profile.get_settings()
input_map = model_profile.get_input_mapping()

In [ ]:
# Make sure we're using the hardware as best we can...

model_settings.BUILD_SETTINGS.BUILD_STR = model_settings.CONFIG_OPTIONS.BUILD_STRINGS.ICC_RAIJIN

In [ ]:
# Now get the model itself

model = model_profile.get_model(model_settings)

In [ ]:
# Examine the input mapping to ensure that the remote paths are as expected

print(input_map.tmin_f)
print(input_map.f_tree_grid)

## Build spec dict

Assemble above settings into specification dictionary

In [ ]:
# Assign a job name; our files and paths will be derived from this
job_name = 'remote_awralv6_test'

In [ ]:
'''
User specifiable calibration description
'''
cal_spec = {}
cal_spec['optimizer_spec'] = optimizer_spec
cal_spec['objective_spec'] = objective_spec
cal_spec['extent_map'] = extent_map
cal_spec['run_period'] = run_period
cal_spec['model'] = model
cal_spec['node_mapping'] = input_map
cal_spec['logfile'] = '%s.h5' % job_name




In [ ]:
'''
Add our raijin helpers;  these will ensure the calibration logfile is written to JOBFS during the run, then copied back,
providing better IO performance for this sort of task

https://opus.nci.org.au/display/Help/What+is+the+jobfs+filesystem.+How+or+when+do+I+use+it

'''

from awrams_user.calibration.raijin_support import prerun_raijin,postrun_raijin

cal_spec['prerun_action'] = callable_to_funcspec(prerun_raijin)
cal_spec['postrun_action'] = callable_to_funcspec(postrun_raijin)

## Launching the remote job

In [ ]:
# RemotePBSManager will be our link to the outside world
# We will build a job for it using cal_spec_to_remote_job

from awrams.cluster.support import RemotePBSManager
from awrams.calibration.cluster import cal_spec_to_remote_job

In [ ]:
#Number of whole nodes to use on Raijin
node_count = 2
#PBS walltime
walltime = '1:00:00'
# The working directory
# You'll need to modify this to your own valid path on raijin
remote_path = '/short/project/username/testjobs/%s' % job_name

In [ ]:
rspec = cal_spec_to_remote_job(cal_spec, job_name, remote_path, node_count, walltime, remote_settings)

In [ ]:
rspec

In [ ]:
pbsman = RemotePBSManager(remote_settings)

In [ ]:
# We can easily sync our user files (config, user code etc)
pbsman.sync_user_files()

In [ ]:
job = pbsman.submit_job_from_spec(rspec)

In [ ]:
pbsman.qstat()

## Job chaining - add a simulation

In [ ]:
# Use the SimulationServer to build a remote job, similarly to running a local simulation

from awrams.simulation.server import SimulationServer

In [ ]:
# Reset the active system profile; we are about to need some local paths again

config_manager.set_active_system_profile()

In [ ]:
sim_extent = extents.get_default_extent()
sim_period = dt.dates('2009-2011')

In [ ]:
# We need the same set of PBS information as for a cal run;

sim_job_name = 'simtest_remote'
#Number of whole nodes to use on Raijin
node_count = 2
#PBS walltime
walltime = '00:20:00'


In [ ]:
# Just like a local sim run, we build an output map
from awrams.simulation.support import build_output_mapping

# We'll reuse the remote_path from our calibration run to keep everything in one place
outpath = remote_path + '/sim_results/'

save_vars = ['qtot','ss','sd','s0_hrusr','s0_hrudr']

# If you want to reinitialise a run from existing states, they will need to be saved at 64bit resolution.
# We will use the 'save_states_freq' argument to create 'snapshots' of states on a monthly basis so you don't have
# to write too much data to disk...
# We'll use mode 'w' here to overwrite any existing data

output_map = build_output_mapping(model, outpath, mode = 'w', save_vars = save_vars, save_states_freq = 'M')

# The updated output map contains write_to_annual_ncfile nodes for the variables we specified above, 
# as well as write_to_ncfile_snapshot for the states
# It's possible to further manipulate this map directly, but in most cases you won't need to

output_map

In [ ]:
# We want this simulation to use the newly generated parameters from our calibration run.
# Because we don't know what they are yet, we can't set the values directly; use
# nodes.parameter_from_calibration_results

# Get the calibration results filename
cal_results_file = os.path.join(remote_path, cal_spec['logfile'])

# Get a list of a parameter keys
param_keys = [k for k,v in input_map.items() if v.node_type == 'parameter']

# Set the new values
for k in param_keys:
    input_map[k] = nodes.parameter_from_calibration_results(cal_results_file, k)

In [ ]:
sim = SimulationServer(model,remote_settings)

In [ ]:
sim_jobspec = sim.get_remote_job(sim_job_name, remote_path, node_count, walltime,\
                                 input_map, output_map, sim_period, sim_extent)

In [ ]:
sim_jobspec

In [ ]:
# Submit the sim job using our existing job as a dependancy;
# the sim job will not run until this job completes successfully
sim_job = pbsman.submit_job_from_spec(sim_jobspec, dependencies = job)

In [ ]:
pbsman.qstat()

In [ ]:
# PBSJobs have methods for examining the output and error streams during and after the run

sim_job.get_output()

## Building a custom PBS job

In [ ]:
from awrams.cluster.support import build_custom_pbs_file, RemoteJobSpec

In [ ]:
# This is not a real python module; just an example of how to build a job

custom_task_str = 'python3 -m awrams_user.example.test ./sim_results'

In [ ]:
print(build_custom_pbs_file('postproc',remote_path,'00:20:00',remote_settings, custom_task_str,ncpus=1,mem='1gb'))

In [ ]:
# Now build a RemoteJobSpec in order to submit the job
# The PBS file will always be built using the job name supplied to build_custom_pbs_file

custom_spec = RemoteJobSpec(remote_path, 'postproc.pbs')

In [ ]:
custom_spec